# Week3 Assignment: Segmenting and Clustering Neighborhoods in Toronto

Import libraries 

In [1]:
import pandas as pd 
print("libraries imported")

libraries imported


Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url,header=0)[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Rename the columns of the dataframe

In [3]:

df.columns=["PostalCode", "Borough","Neighborhood"]
df.columns 


Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

drop the missing Borough value, in this case is 'Not assigned' values

In [4]:
df = df[df.Borough != "Not assigned"]
df['Borough'].str.contains("Not assigned").any() # check if is all the values are droped

False

check if there is any missing values in the Neighborhood column.Even tough the ansewer is no, we choose to provide the code for replacing the missing values wih the Borogh values

In [5]:
print(df.isnull().sum().any())
df['Neighborhood'] = df['Neighborhood'].fillna(df["Borough"])

False


We check for the duplicated values. The dataframe doesn't contain any duplicated Postal Codes. But we observe that instead of the format suggested in the instructions (using the comma seperation), the dataframe uses the slash as sepration. 

In [6]:
df.duplicated('PostalCode').any() # check for the duplicated values
df['Neighborhood']=df['Neighborhood'].str.replace('/', ',') #replace '/' by ','
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


we chose also to provide the code to combine the rows with the same postal code into one row with the neighborhoods separated with a comma 

In [7]:
df=df.groupby(['PostalCode',"Borough"],sort=False)['Neighborhood'].apply(lambda x: ' , '.join(x)).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


check the shape of the final dataframe. 

In [8]:
df.shape

(103, 3)

Import geolocalisation data and store it in coords dataframe 

In [9]:
coords=pd.read_csv('/media/bsimaa27/Data/Data_science/IBM_Certification/ibm_capstoneproject/basma/Geospatial_Coordinates.csv')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the coordinates dataframe and the dataframe with neighborhood informations 

In [10]:
df=df.set_index('PostalCode').join(coords.set_index('Postal Code'), sort=True).reset_index()
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
